In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
dataPath = '../myMediaPipe/trainingData/210316_1115'

In [3]:
!ls $dataPath

ls: ../myMediaPipe/trainingData/210316_1115: No such file or directory


In [ ]:
labelsFile = os.path.join(dataPath, "labels.csv")
labels = pd.read_csv(labelsFile,header=None)
print(labels)

In [ ]:
preData  = {}
for label in labels.itertuples():
    trainingFile = os.path.join(dataPath,label[2] + ".csv")
    preData[label.Index] = pd.read_csv(trainingFile,header=None)
    print("Loading file:", trainingFile, "\tNumber of Rows:" , len(preData[label.Index].index))
    print(label.Index)


In [ ]:
# cleaning
for key in preData.keys():
    print("Record #:", key, "\thas:",len(preData[key].index) % 21, "incomplete captures")
    preData[key].drop(preData[key].tail(len(preData[key].index) % 21).index,inplace=True)

In [ ]:
# balance data
recordsPerClass = len(preData[0].index) #initial value
#Calculate smallest class
for key in preData.keys():
    if recordsPerClass > len(preData[key].index): 
        recordsPerClass=len(preData[key].index)
    #print(recordsPerClass)

#Trimming
for key in preData.keys():
    preData[key] = preData[key].head(recordsPerClass)
    print(len(preData[key].index))

In [ ]:
# Builds training and label array

In [ ]:
numTrainRows, numTestRows = 0,0
numLandmaks = 21
numCols = numLandmaks*2


def countRows(dataDict):
    rowsCount=0
    for key in dataDict.keys():
        rowsCount += len(dataDict[key].index)
    return rowsCount

numRows = countRows(preData)
#numTestRows = countRows(testData)

numTransRows = int(numRows / numLandmaks)
transData = np.zeros((numTransRows,numCols),preData[1][4].dtypes)
transLabels = np.zeros((numTransRows),np.int16)

#numTestRows = int(numTestRows / numLandmaks)
#testingData = np.zeros((numTestRows,numCols),preData[1][4].dtypes)
#testingLabels = np.zeros((numTestRows),np.int16)

print("Number of Rows in total:", numTransRows)  
print(transData.shape, transData.dtype)
print(transLabels.shape, transLabels.dtype)

#print(testingData.shape, testingData.dtype)
#print(testingLabels.shape, testingLabels.dtype)


In [ ]:
def transposeInterleave(a, b):
    c = np.empty((a.size + b.size,), dtype=a.dtype)
    c[0::2] = a
    c[1::2] = b
    return c

def dataTransp(dataDict,processedData,processedLabels):
    j = 0
    for key in dataDict.keys():
        for i in range(0, len(dataDict[key].index), numLandmaks):  
            #print(dataDict[1].iloc[i : i + numLandmaks,3:5])
            a= np.ravel(np.asarray(dataDict[key].iloc[i : i + numLandmaks,3:4]))
            b= np.ravel(np.asarray(dataDict[key].iloc[i : i + numLandmaks,4:5]))
            #print("j:",j)
            processedData[j] = transposeInterleave(a,b)
            processedLabels[j] =labels.iloc[key,2]
            j+=1
     # retun True

dataTransp(preData,transData,transLabels) 

print(transData[-1])
print(transLabels[-1])

In [ ]:
# Labels to one hot

In [ ]:
from keras.utils import to_categorical
print(transLabels.shape)
transLabels = to_categorical(transLabels)
print(transLabels.shape)
print(transLabels[-1])

In [ ]:
# Set apart Test Data

In [ ]:
from sklearn.model_selection import train_test_split
testDataRatio= 0.2

X_train, X_test, y_train, y_test = train_test_split(transData,transLabels, test_size=testDataRatio)
print( "\tTraining records:", np.shape(X_train)[0], "\tTest records:", np.shape(X_test)[0])

In [ ]:
# model definition

In [ ]:
from keras import models
from keras import layers
from keras import backend as K
K.clear_session()

model = models.Sequential()
model.add(layers.Dense(64, activation='relu', input_shape=(numCols,)))
model.add(layers.Dense(42, activation='relu'))
model.add(layers.Dense(len(labels.index), activation='softmax'))

In [ ]:
# model compile

In [ ]:
model.compile(optimizer='rmsprop',
loss='categorical_crossentropy',
metrics=['accuracy'])

In [ ]:
# training

In [ ]:
history = model.fit(X_train,
                    y_train,
                    epochs=40,
                    #batch_size=512,
                    validation_split=0.2)

In [ ]:
# Plotting the training and validation loss

In [ ]:
import matplotlib.pyplot as plt
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# Plotting the training and validation accuracy

In [ ]:
plt.clf()
acc = history.history['acc']
val_acc = history.history['val_acc']
plt.plot(epochs, acc, 'go', label='Training acc')
plt.plot(epochs, val_acc, 'g', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# Evaluate the model

In [ ]:
loss, acc = model.evaluate(X_test, y_test, verbose=0)
print('\nTesting loss: {}%, acc: {}%\n'.format(round(loss*100,2),round(acc*100,2)))

In [ ]:
# TODO: Model optimization

In [ ]:
# saving
model.save('gestures002.h5')

In [ ]:
# To tflite

In [ ]:
import tensorflow as tf
print(tf.version)

converter = tf.lite.TFLiteConverter.from_keras_model_file( 'gestures001.h5') 
liteModel = converter.convert()

open("gestures002.tflite", "wb").write(liteModel)

#file = open( 'gestures001.tflite' , 'wb' ) 
#file.write( liteModel )